<style>
    .titulo-violeta {
        color: #c485ff;
    }
</style>

<h1 class="titulo-violeta">TP Convolutional Neural Networks</h1>

## <span style="color:lightblue">Avanzado IA</span>

### <span style="color:lightblue">Tp hecho por Benjamin Kapustiansky y Oliver Safdieh</span> 

## Problematica a resolver:
Se necesita un modelo de IA para poder identificar zonas directamente afectadas por una inundación.

## Solución propuesta:
Mediante un modelo de CNN, nosotros logramos identificar las zonas afectadas con un modelo Unet el cual segmenta la imagen para poder desglosarla y entenderla de una mejor manera.

## <span style="color:lightblue">Libraries</span>

In [1]:
# Estas líneas de código nos permiten instalar opencv desde Jupyter Notebook, que vamos a necesitar para importar 'cv2'
import sys
!{sys.executable} -m pip install opencv-python --user


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install tensorflow seaborn numpy pandas matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.image as tfi
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D,  MaxPool2D, UpSampling2D, concatenate, Activation
from tensorflow.keras.layers import Layer, Input, Add, Multiply, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
    
import seaborn as sns
sns.set_style('darkgrid')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

print(tf.__version__)

2.17.0


## <span style="color:lightblue">Dataset</span>

In [4]:
archivo = "metadata.csv"

# Leer datos del archivo
flood : pd.DataFrame = pd.read_csv(archivo)
flood

,Image,Mask
0,0.jpg,0.png
1,1.jpg,1.png
2,2.jpg,2.png
3,3.jpg,3.png
4,4.jpg,4.png
...,...,...
285,1083.jpg,1083.png
286,1084.jpg,1084.png
287,1085.jpg,1085.png
288,1086.jpg,1086.png


## Function to create data

In [5]:
def create_data(data_dir):
    image_paths = []
    mask_paths = []
    
    folds = sorted(os.listdir(data_dir))
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        if fold in ['image', 'Image', 'images', 'Images', 'IMAGES']:
            images = sorted(os.listdir(foldpath))
            for image in images:
                fpath = os.path.join(foldpath, image)
                image_paths.append(fpath)

        elif fold in ['mask', 'Mask', 'masks', 'Masks', 'MASKS']:
            masks = sorted(os.listdir(foldpath))
            for mask in masks:
                fpath = os.path.join(foldpath, mask)
                mask_paths.append(fpath)
        else:
            continue
        
    return image_paths, mask_paths

## Function to read images

In [6]:
# function to read an image
def load_image(image, SIZE):
    return np.round(tfi.resize(img_to_array(load_img(image)) / 255., (SIZE, SIZE)), 4)

# function to read multiple images
def load_images(image_paths, SIZE, mask=False, trim=None):
    if trim is not None:
        image_paths = image_paths[:trim]
    
    if mask:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 1))
    else:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 3))
    
    for i, image in enumerate(image_paths):
        img = load_image(image, SIZE)
        if mask:
            images[i] = img[:, :, :1]
        else:
            images[i] = img
    
    return images